In [ ]:
# # !pip install pytesseract
# !pip install easyocr
# !pip install googletrans==3.1.0a0
# !pip install chinese

In [2]:
import os
# import pytesseract
import easyocr
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from googletrans import Translator
from chinese import ChineseAnalyzer
from google.colab.patches import cv2_imshow
from PIL import Image, ImageDraw, ImageFont

try: 
    from PIL import Image
except ImportError:
    import Image

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
def text_wrap_by_width(text, font, max_width, trans_lang, analyzer=None):
    """Wrap text basing on specified width. 
        text: text to wrap
        font: font of the text
        max_width: width of the bounding box to split the text with
        trans_lang: от языка зависит, как разбивать текст на слова
        analyzer:  для разделения китайского текста на слова
        return: list of sub-strings """

    lines = []
    
    # If the text width is smaller than the image width, then no need to split
    # just add it to the line list and return
    # font.getsize(text)[0] - width, font.getsize(text)[1] - height

    if font.getsize(text)[0] <= max_width:
        lines.append(text)
    else:
        #split the line by spaces to get words
        if trans_lang == 'zh-cn':
          words = analyzer.parse(text).tokens()
        else:
          words = text.split(' ')   # список слов

        # print('words: ', words)
        i = 0   # счетчик слов
        # append every word to a line while its width is shorter than the image width
        while i < len(words):
            # print(' len(words) ',  len(words))
            line = ''
            while i < len(words) and font.getsize(line + words[i])[0] <= max_width:
                if trans_lang == 'zh-cn':  # если язык китайский, то не нужно добавлять пробелы между словами
                  line = line + words[i]
                else:
                  line = line + words[i]+ " "
                i += 1
            if not line:
                line = words[i]
                i += 1
            lines.append(line)
    return lines

In [5]:
def text_wrap_by_h_w(font_path, text, max_width, max_height, trans_lang):
  ''' Define suitable size of the font, considering given font, height and width of the bounding box '''

  # т.к. в китайском нет пробелов, то разделение текста по пробелам не сработает
  if trans_lang == 'zh-cn':  # если язык, на который переведен текст, - китайский
    analyzer = ChineseAnalyzer()   # то инициализируем analyzer, который поможет разбить текст на слова
    # выполним инициализацию в этой функции, до вызова функции text_wrap_by_width, т.к. инициализация занимает много времени
    # и если делать это в text_wrap_by_width, которая вызывается много раз, то это будет занимать достаточно много времени
  else: 
    analyzer = None
    
  fontsize = 5
  font = ImageFont.truetype(font=font_path, size=fontsize)  # font of the text
  line_height = font.getsize('hg')[1]    # значение высоты строки (измеряем высоту букв h,g)
  wrapped_text = text_wrap_by_width(text, font, max_width, trans_lang, analyzer)
  
  while (line_height * len(wrapped_text)) <= max_height:
    fontsize += 1
    font = ImageFont.truetype(font=font_path, size=fontsize)  # font of the text
    wrapped_text = text_wrap_by_width(text, font, max_width, trans_lang, analyzer)
    line_height = font.getsize('hg')[1]

    if (line_height * len(wrapped_text)) > max_height:
      fontsize -= 1
      font = ImageFont.truetype(font=font_path, size=fontsize)  # font of the text
      wrapped_text = text_wrap_by_width(text, font, max_width, trans_lang, analyzer)
      line_height = font.getsize('hg')[1]
      break

  return wrapped_text, font, line_height

In [6]:
def get_average_color(image, elem):
  '''Get average color on the borders of bounding boxes'''

  sum_all_borders = [0,0,0]
  number_of_pixels =  2*(elem[0][2][0] - elem[0][0][0]) + 2*(elem[0][2][1] - elem[0][0][1])

  y = elem[0][0][1]-2 if elem[0][0][1]-2 > 0 else 1   # проверка, не выходит ли за границы изображения
  for x in range(elem[0][0][0], elem[0][2][0]):
    for i in range(3):
      sum_all_borders[i] += image.getpixel((x, y))[i]  # get the sum of pixels above the top border of bounding box

  y = elem[0][2][1]+2 if elem[0][2][1]+2 < image.size[1] else image.size[1]-1
  for x in range(elem[0][0][0], elem[0][2][0]):
    for i in range(3):
      # print(elem[0][2][1], image.size[1], image.size[0], y)
      sum_all_borders[i] += image.getpixel((x, y))[i]  # get the sum of pixels under the bottom border of bounding box

  x = elem[0][0][0]-2 if elem[0][0][0]-2 > 0 else 1
  for y in range(elem[0][0][1], elem[0][2][1]): 
    for i in range(3):
      sum_all_borders[i] += image.getpixel((x, y))[i]   # get the sum of pixels on the left border of bounding box

  x = elem[0][2][0]+2 if elem[0][2][0]+2 < image.size[0] else image.size[0]-1
  for y in range(elem[0][0][1], elem[0][2][1]): 
    for i in range(3):
      sum_all_borders[i] += image.getpixel((x, y))[i]  # get the sum of pixels on the right border of bounding box 

  average_color = tuple([int(elem/number_of_pixels) for elem in sum_all_borders])

  return average_color

In [17]:
def text_inpainting(image_path, font_path, text_from_image, translated_text, source_lang, trans_lang):
  
  image_name = image_path.split('/')[-1]  # split by '/' and take the last element of the list, which is the name of the image

  image = Image.open(fp=image_path)
  img_draw = ImageDraw.Draw(im=image)

  # img_draw.rectangle([(0, 0), (image.size[0]-0.5, image.size[1]-0.5)], fill ="white", outline ="red")  # boundaries of the image
  # img_draw.rectangle([(0, 0), (image.size[0]-0.5, image.size[1]-0.5)], outline ="red")

  for i, elem in enumerate(text_from_image): 
    
    # print('\n', elem[0], translated_text[i].text, '\n')

    text = translated_text[i].text
    
    # elem[0] - [[x_left, y_up], [x_right, y_up], [x_right, y_down], [x_left, y_down]]; 
    # elem[0][2] - [x_right, y_down], elem[0][0] - [x_left, y_up];     
    bb_width = elem[0][2][0] - elem[0][0][0]    # elem[0][2][0] - elem[0][0][0] = x_right - x_left
    bb_height = elem[0][2][1] - elem[0][0][1]    # elem[0][2][1] - elem[0][0][1] = y_down - y_up

    lines, font, line_height = text_wrap_by_h_w(font_path, text, bb_width, bb_height, trans_lang)

    x = elem[0][0][0]
    y = elem[0][0][1]

    # img_draw.rectangle([tuple(elem[0][0]), tuple(elem[0][2])], fill ="white", outline ="green")   # boundaries of the text in image
    
    background_color = get_average_color(image, elem)
    img_draw.rectangle([tuple(elem[0][0]), tuple(elem[0][2])], fill = background_color)
    

    # Define color of the text in contrast with background color
    if background_color > (127, 127, 127):   # если ближе к белому
      text_color = (background_color[0]-110, background_color[1]-110, background_color[2]-110)
    else:   # если ближе к темному
      text_color = (background_color[0]+100, background_color[1]+100, background_color[2]+100)

    # (0,0,0) - black
    # (255, 255, 255) - white
    for line in lines:
        img_draw.text((x,y), line, fill=text_color, font=font)
        y = y + line_height    # update y-axis for new line 

  # image.show()
  if source_lang == 'zh-cn':  # т.к. папка называется ch_sim, если оставить source_lang = 'zh-cn', путь '/content/drive/MyDrive/Универ/ocr/images/{source_lang}' не будет существовать
   source_lang = 'ch_sim'
  # image_name[image_name.index("."):] - расширение файла после точки
  # image_name[:image_name.index(".")] - имя файла без расширения
  new_image_name = f'{image_name[:image_name.index(".")]}_{source_lang}-{trans_lang}_translated{image_name[image_name.index("."):]}'
  image.save(f'/content/drive/MyDrive/Универ/ocr/images/{source_lang}/{source_lang}_translated/{new_image_name}')
                                                                 # {source_lang}-{trans_lang}_translated_{image_name}')
  # image.save(new_image_name)

In [15]:
def ocr(image_path, source_lang):
  # будет распознавать английский и другой переданный язык (поддерживает распознавание нескольких языков одновременно, английский совместим со всеми)
  reader = easyocr.Reader(['en', source_lang], gpu=True)  
  text_from_image = reader.readtext(image_path, detail = 1, paragraph = True, 
                                    # width_ths = 1.8
                                    )
  return text_from_image

In [9]:
def text_translation(text_from_image, source_lang, trans_lang):

  translator = Translator()

  translated_text= []
  for elem in text_from_image:
    translated_text.append(translator.translate(elem[1], src=source_lang, dest=trans_lang))  # 'zh-cn': 'chinese (simplified)', 'de': 'german', 'en': 'english', 'ru': 'russian'

  return translated_text

In [10]:
def image_translation(image_path, source_lang, trans_lang):
  
  text_from_image = ocr(image_path, source_lang)
  # print(text_from_image)
  print('OCR is completed!')

  if source_lang == 'ch_sim':
    source_lang = 'zh-cn'
  
  translated_text = text_translation(text_from_image, source_lang, trans_lang)
  
  # for trans in translated_text:
    # print(trans.origin)
    # print(trans.text)

  print('Text translation is completed!')

  # Save recognized text and its translation
  txt_name = f'{image_path[:image_path.rindex("/")]}/txt/{image_path.split("/")[-1]}.txt'
  with open(txt_name, 'a') as f:

    if os.path.getsize(txt_name) == 0:  # если изображение переводится первый раз (файл пустой), то записываем все
      f.write('Распознанный текст: \n')

      # создание списка строк, т.к. f.write должен принимать на вход строку; .join объединяет в строку только список строк, а text_from_image - массив массивов
      strings = [str(elem) for elem in text_from_image]  
      f.write('\n'.join(strings))   # сохраним распознанный текст
      
      f.write('\n\nПеревод: \n')
      f.write(f'{source_lang} - {trans_lang}\n')  # с какого языка на какой выполнен перевод
      
      # создание списка строк, т.к. f.write должен принимать на вход строку; .join объединяет в строку только список строк
      # а translated_text - массив объектов <googletrans.models.Translated>
      trans_text = [trans.text for trans in translated_text]
      f.write('\n'.join(trans_text))   # сохраним перевод распознанного текста

    else:  # если в файл уже записан какой-то перевод, то просто добавляем новый

      f.write(f'\n\n{source_lang} - {trans_lang}\n')  # с какого языка на какой выполнен перевод
      
      # создание списка строк, т.к. f.write должен принимать на вход строку; .join объединяет в строку только список строк
      # а translated_text - массив объектов <googletrans.models.Translated>
      trans_text = [trans.text for trans in translated_text]
      f.write('\n'.join(trans_text))   # сохраним перевод распознанного текста


  # font_path = '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf'
  font_path = '/content/drive/MyDrive/Универ/ocr/TIMES.TTF'
  if trans_lang == 'zh-cn':
    font_path = '/content/drive/MyDrive/Универ/ocr/SIMSUN.TTC'   # шрифт для китайского языка


  text_inpainting(image_path, font_path, text_from_image, translated_text, source_lang, trans_lang)
  print('Text inpainting is completed!')

In [ ]:
common_path = '/content/drive/MyDrive/Универ/ocr/'

# с каждого языка на все другие
# for source_lang in sorted(['ru', 'de', 'ch_sim', 'en']):
#   for trans_lang in sorted(['de', 'zh-cn', 'en', 'ru']):
#     if source_lang != trans_lang and not (source_lang == 'ch_sim' and trans_lang == 'zh-cn'):   # чтобы исходный язык и тот, на который нужно перевести, не были равны

# чтобы перевести все изображения в папке common_path+f'images/{source_lang}' с одного языка на все другие
source_lang = 'ru'
for trans_lang in sorted(['de', 'zh-cn', 'en']):   
  
  for image_path in tqdm(sorted(os.listdir(common_path+f'images/{source_lang}'))):

    if not os.path.isdir(common_path + f'images/{source_lang}/' + image_path):
      image_path = common_path + f'images/{source_lang}/' + image_path

      print('\n', image_path, source_lang, trans_lang)
      image_translation(image_path, source_lang, trans_lang)

  0%|          | 0/18 [00:00<?, ?it/s]WARNING:easyocr.easyocr:Using CPU. Note: This module is much faster with a GPU.



 /content/drive/MyDrive/Универ/ocr/images/ru/rus.jpg ru de
OCR is completed!
Text translation is completed!


 11%|█         | 2/18 [00:22<03:03, 11.48s/it]WARNING:easyocr.easyocr:Using CPU. Note: This module is much faster with a GPU.


Text inpainting is completed!

 /content/drive/MyDrive/Универ/ocr/images/ru/rus1.jpg ru de
OCR is completed!
Text translation is completed!


 17%|█▋        | 3/18 [01:06<06:12, 24.85s/it]WARNING:easyocr.easyocr:Using CPU. Note: This module is much faster with a GPU.


...

Text inpainting is completed!

 /content/drive/MyDrive/Универ/ocr/images/ru/test_rus8.jpg ru zh-cn
OCR is completed!
Text translation is completed!


 89%|████████▉ | 16/18 [07:59<00:52, 26.46s/it]WARNING:easyocr.easyocr:Using CPU. Note: This module is much faster with a GPU.


Text inpainting is completed!

 /content/drive/MyDrive/Универ/ocr/images/ru/test_rus9.jpg ru zh-cn
OCR is completed!
Text translation is completed!


100%|██████████| 18/18 [08:37<00:00, 28.76s/it]

Text inpainting is completed!
